In [ ]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Imports

In [231]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

matplotlib.rcParams.update({'font.size': 21})
# matplotlib.rcParams["figure.dpi"] = 300

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from scipy.optimize import curve_fit
from scipy.signal import savgol_filter, butter

In [7]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [8]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [102]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run ../../Functions/0.2/data_fct_Adrien_point_by_point

In [12]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichier

In [13]:
## GUI for data loading
# Select one data file all the way down to the directories
# Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG.dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : D:/CIMLuser/adrien/Simulations/20200610/DC09_RF11/Try00
> Filename : Temp_SimuType0_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [2, 11, 18, 30, 39, 49]
> slashcond | 4
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 1


In [14]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC09_RF11']
[4]
['DC', []]
> condition names ['DC', 'RF']
> number of points 1
000 > DC09_RF11
> N_ions = 1024
> e_GMol = 50


In [15]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir, points_and_coord,condition_parameters,slash_cfg,mode='GMol') # stopping_power

data0 = data_name[0]
data2 = data_name[1]
data4 = data_name[2]

PMvar = PMandT[0]
Tvar = PMandT[1]

deltaEc = Gmol_data[0]
deltaEcRel = Gmol_data[1]
t_c = Gmol_data[2]

> Points | 1
> Simulations pour chaque point | ['Try00']
Hello
['Temp_SimuType0_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG.dat'
 'Temp_SimuType2_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuType4_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuType6_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG.dat'
 'Temp_SimuType6_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG50eV.dat']
0
Point n° 0
00 - 00  >  {}\D:/CIMLuser/adrien/Simulations/20200610\DC09_RF11\Try00
Time elapsed:  0.8573808000000014 s
Time elapsed:  0.014289680000000023 m


In [16]:
# Where SNR for eeach try in each  are compared to a threshold
# If SNR > some_threshold , we count it as a detection
# Then for each point the ratio number_of_detection/number_of_try is calculated
# This detection efficiency 

SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))
trig_detection = np.zeros((shapevar))
for i in range(len(points_and_coord)):
    SNR = Gmol_data[2][i]
    for k,m in enumerate(SNR):
        if m > SNR_trig_threshold:
            trig_detection[i,k] = 1

print(mean(trig_detection,axis=1))

print(len(trig_detection))

[1.]
1


In [17]:
# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

1.0


# Plots
Nuage d'ions (XYZ)

## Visualise one specific point, and one case

In [18]:
# Choose point to plot

plot_point = 0
print(list(points_and_coord.keys())[plot_point]) # point condition
plot_try = 0
plot_subdir = all_subdir[plot_point+plot_try%13] # point address
print(plot_subdir)

DC09_RF11
D:/CIMLuser/adrien/Simulations/20200610\DC09_RF11\Try00


In [ ]:
# plot ion cloud XYZ

print(plot_subdir)
onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
file_name = '{}/xva{}'.format(plot_subdir,sort(onlyfiles)[0].strip('.dat')[4:])
plot_XYZ(file_name,fig_name='XYZ_point'+str(plot_point)+'_try'+str(plot_try),fig_title='XYZ')

In [19]:
# Plot T and fluo PM

onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
ta, te, ti, to, tu = plot_T_and_PM_Init_Inje_Evol( plot_subdir+'/',sort(onlyfiles)[0].strip('.dat')[4:],1,'coucou' )

# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

# Température avec pas de temps réaliste

In [227]:
# Plot only T with simulation time step
fname = 'only T'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
ax.plot(ta*1000,ti)
ax.grid()

ax.set_xlabel('[ms]')
ax.set_ylabel('[cts/500ns]')
tight_layout()
 
# savefig('ex_simu_clic_tr6_dc4_rf5_zoom.png',dpi=300)

In [ ]:
# Plot only T with custom time step (realistic PM behaviour)

time_step = 1e-3 # s
custom_pts = len(ta)*(ta[1]-ta[0])/time_step
print(custom_pts)
somme_pts = int( round( time_step/(ta[1]-ta[0]) ) )
print(somme_pts)

custom_t = np.linspace(1,ceil(max(ta)/time_step),custom_pts+1)
custom_t_offset = max(custom_t) - max(ta)/time_step
custom_t = custom_t - custom_t_offset
custom_fluo = np.zeros( int(ceil(custom_pts)) )

for j in range( int(ceil(custom_pts))-1 ):
    temp = []
    print(j)
    print(somme_pts*j,somme_pts*(j+1)-1)
    for k in range(somme_pts*j,somme_pts*(j+1)):
        temp.append(ti[-k-1])
    temp = sum(temp)
    custom_fluo[-1-j] = temp/max(ti)/15

fname = 'only T custom T step'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
ax.plot(ta*1000,ti,zorder=10,label='500 ns')
# for j in range( int(ceil(custom_pts)) ):
#     ax.plot(ta[::-1][somme_pts*j:somme_pts*(j+1)]*1000,ti[::-1][somme_pts*j:somme_pts*(j+1)])
ax.scatter(custom_t[1:],custom_fluo[1:]/max(custom_fluo)*max(ti),s=200,marker='+',lw=5,edgecolor='black',zorder=100,label='1 ms')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
ax.grid()


ax.set_xlim(1.1,5.98)
ax.set_xlabel('[ms]')
ax.set_ylabel('[cts]')
ax.legend(title='Time step')
tight_layout()
 
# savefig('ex_simu_clic_tr6_dc4_rf5_zoom.png',dpi=300)

## Data visualisation for all points (SNR)

In [ ]:
# Voltages associated with each condition
# In names and files each voltage is associated with numbers
# For instance on epoint is labeled DC03_RF18
# You need to know which voltage is DC03 and RF18

# You have to look on the Fortran code, example :
#    !dec$ elseif defined(Vrf18)
#    double precision  , parameter :: V_rf  = 7.179d1
#    character(len=100), parameter :: str_extra3 = 'q=0.666'
# Here RF18 is 71.79 V, q = 0.666

Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]
len(Udc),len(Urf)

In [ ]:
# A 3D visualisation of SNR for each point, each run
# Compulsory because this is where SNR is put in shape (under the name Z)

ax = 0
ay = 0
az, qx, wx, wy, wz = [],[],[],[],[]

fname = '3D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111, projection='3d')

# fig1 = plt.figure(num=fname,clear=True)
# ##### PLOT DFluo et Proba détection
# ax1 = fig1.add_subplot(111)
# ax2 = ax1.twinx()
# clf()

l = 0
plot_Udc = []
plot_Urf = []
Z = []
for i,j in enumerate(points_and_coord):
    plot_Udc.append(Udc[int(points_and_coord[j][0])])
    plot_Urf.append(Urf[int(points_and_coord[j][1])])
    Z.append(Gmol_data[2][i])
            # ax.set_aspect('equal')
Z = np.array(Z)
print(shape(plot_Udc),shape(plot_Urf),shape(Z))

for k in range(13):
    ax.scatter(xs=plot_Udc,ys=plot_Urf,zs=Z[:,k],color='xkcd:azul',marker='o',s=10)
#             ax.plot(xs=r_LC_clip[0],ys=r_LC_clip[2],zs=r_LC_clip[1],marker='o',ms=0.45,ls='',color='xkcd:blue with a hint of purple')

#             ax.view_init(elev=18, azim=55)
#             ax.set_xlim(-3e-1,3e-1)
#             ax.set_ylim(-3e-1,3e-1)
#             ax.set_zlim(-3e-1,3e-1)

#             ax.text2D(0.05, 0.85, f'$U_{{ RF }}$ =  {Urf} V\n$U_{{DC}}$ = {Udc[j]} V', transform=ax.transAxes)
ax.set_xlabel('\n$U_{DC}$ [V]')
ax.set_ylabel('\n$U_{RF}$ [V]')
ax.set_zlabel('\n SNR ')

#         ax.plot(np.ones(100)*Udc[dc], np.ones(100)*Urf[rf], zs=linspace(0,0.8,100),color='black',ls=':',lw=1)

# X = [Udc for hhh in range(elem_1+1)]
# Y = [Urf for smeuiofh in range(elem_2-1)]
# # X, Y = np.meshgrid(X, Y)
# Z = asarray(transpose(mean_fluo))
# Z = np.meshgrid(Z)

# # surf = ax.plot_wireframe(transpose(X), Y, Z)
# surf = ax.plot_surface(transpose(X), Y, Z, rstride=1, cstride=1,
#                 cmap='viridis', edgecolor='black',alpha = 0.35)
# #         ax.errorbar(Udc[dc],mean_fluo[],yerr=prob_detect_xerr,xerr=None,
# #              elinewidth=2.5,capsize=None,color='xkcd:yellow orange',
# #              fmt=':+',ms=12,mew=3.5,mfc='None',mec='xkcd:yellow orange')
# cbar = plt.colorbar(surf)
# cbar.ax.set_ylabel(r'$\Delta F / F$ moyen ('+str(elem_0)+' essais)', rotation=90)


fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
# subplots_adjust(hspace=0.015)

# savefig(fname,dpi=300)

## 2D plot

In [ ]:
for i,j in enumerate(points_and_coord):
    print(j)
    print(points_and_coord[j][0],points_and_coord[j][1])

In [ ]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(Udc[int(points_and_coord[h][0])])
    y.append(Urf[int(points_and_coord[h][1])])
    z.append(mean(trig_detection[g]))
    
fname = '2D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][:-15],fontsize=12)

im = ax.scatter(x,y,
        marker='o',s=100,edgecolor='black',
        c = z, cmap='rainbow')
ax.set_xlabel(r'$U_{DC}$ [V]')
ax.set_ylabel(r'$U_{RF}$ [V]')
ax.grid()

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(SNR_trig_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'.png',dpi=300)

In [ ]:
fname = 'art'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)

im = ax.contourf([1,2,3],[1,2,3],[[1,2,3],[4,5,6],[7,8,9]]) # [[nan,2,nan],[4,5,6],[nan,8,nan]]
# for rf in range(1,4):
#     for dd in range(1,4):
#         ax.plot(rf,dd,marker='+',ls='',markersize=10,mew=3,color = 'xkcd:yellow orange')

# ax.set_zlabel(r'$\Delta F / F$')
# im2 = ax.contour(Udc, Urf, transpose(Z), colors='k')
ax.set_xlim(0.95,3.05)
ax.set_ylim(0.95,3.05)
fig.set_size_inches(11,8)
fig.tight_layout()
# savefig(fname+str(40),dpi=300)

## More specific visualisations

In [ ]:
# Choose a specific condition to consider
condition_to_respect = 'RF10'
points_to_plot = [k for k,l in enumerate(list(points_and_coord.keys())) if condition_to_respect in l]
print(points_to_plot)

In [ ]:
# plot delta E Gmol absolu for one RF voltage

SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))

ax = 0
ay = 0
az, qx, wx, wy, wz = [],[],[],[],[]
alpha_dre = zeros(shapevar)
rtoLratio = zeros(shapevar)

fname = 'Fluo_stat_all'
fig1 = plt.figure(num=fname,clear=True)
gs = gridspec.GridSpec(2, 2, width_ratios=[3, 1]) 
##### PLOT DFluo et Proba détection
ax1 = fig1.add_subplot(gs[0])
ax2 = ax1.twinx()
# clf()

DC_index = list(points_and_coord)[0].find('DC')
RF_index = list(points_and_coord)[0].find('RF')
plot_Udc = [Udc[ int(list(points_and_coord)[x][DC_index+2:DC_index+4])] for x in points_to_plot]
plot_Urf = [Urf[ int(list(points_and_coord)[x][RF_index+2:RF_index+4])] for x in points_to_plot]

suptitle(all_subdir[0][:-15]+'  | Urf='+str(plot_Urf[0])+', D=-'+str(Detuning),fontsize=12)

# az.append(8*mkappa*(C_e/m_Ca)*plot_Udc/(d0**2*Omega**2))
# qx.append(-4*(C_e/m_Ca)*plot_Urf/(r0**2*Omega**2))

#     # frequences attendues
#     wx.append(Omega/2*sqrt(qx[l]**2/2+ax-0.5*az[l])/(2*pi))
# #     wy.append(Omega/2*sqrt(qx[l]**2/2+ay-0.5*az[l])/(2*pi))
#     wz.append(sqrt(2*mkappa*C_e*plot_Udc[i]/(m_Ca*d0**2))/(2*pi))
    
for i,j in enumerate(points_to_plot):
    # a,q attendu       
    SNR = Gmol_data[2][j]
    ax1.plot([plot_Udc[i]]*len(SNR),SNR,'o',color='xkcd:azul',markerfacecolor='none')
    
    prob_detect_xerr = sqrt(mean(trig_detection,axis=1)[j]*(1-mean(trig_detection,axis=1)[j]))/sqrt(len(SNR))
    ax2.errorbar(plot_Udc[i],mean(SNR),yerr=prob_detect_xerr,xerr=None,
             elinewidth=1,capsize=None,color='xkcd:yellow orange',
             fmt=':+',ms='8',mew=2,mfc='None',mec='xkcd:yellow orange')
#     annotate('qx={:1.1e}\nUrf={:3.1f}V\nwx={:3.1f}kHz'.format(qx[0],Urf_no_repeat,wx[0]/1000), xy=(2,0.25), xycoords='data',
#         size=10, ha='center', va='top', color='xkcd:azul',
#         bbox=dict(boxstyle='round', fc='none',edgecolor='xkcd:azul'))
    
    l+=1
    
# for k in range(0,len(Udc_no_repeat),3):
#         ax1.annotate('a={:1.1e}\nwz={:3.1f}kHz'.format(az[k],wz[k]/1000), xy=(Udc[k],0.73), xycoords='data',
#         size=8, ha='center', va='top', color='xkcd:azul',
#         bbox=dict(boxstyle='round', fc='none',edgecolor='none')) 
       
ax1.set_ylabel('SNR', color='xkcd:azul')
# ax2.set_ylabel('$P_{Detect}$'+' ('+str(elem_0)+' runs)', color='xkcd:yellow orange')
ax1.tick_params(axis='y', labelcolor='xkcd:azul')
ax2.tick_params(axis='y', labelcolor='xkcd:yellow orange')


ax1.set_ylim(-0.15,3)
ax2.set_ylim(-0.15,3)
ax1.grid()

##### Plot DEnergie et moyenne de DEnergie
renorm = 0.5*m_GM/C_e
ax3 = fig1.add_subplot(gs[2],sharex=ax1)
ax4 = ax3.twinx()
# clf()
for i,j in enumerate(points_to_plot):
    ax3.plot([plot_Udc[i]]*len(deltaEc[j][:]),deltaEc[j][:]*renorm*1000,'o',color='xkcd:azul',markerfacecolor='none')

mean_deltaEc = [mean(deltaEc[j][:])*renorm*1000 for i,j in enumerate(points_to_plot)]
        
ax4.plot(plot_Udc,mean_deltaEc,linestyle='',marker='o',color='xkcd:yellow orange')
ax4.annotate('$E_{GMol}$=50 eV', xy=(1,5), xycoords='data',
    size=10, ha='center', va='top', color='xkcd:azul',
    bbox=dict(boxstyle='round', fc='none',edgecolor='xkcd:azul')) 

ax3.set_xlabel('$U_{DC}$')
ax3.set_ylabel('$\Delta E$ [meV]', color='xkcd:azul')
ax4.set_ylabel('mean $\Delta E$ [meV]', color='xkcd:yellow orange')
ax3.tick_params(axis='y', labelcolor='xkcd:azul')
ax4.tick_params(axis='y', labelcolor='xkcd:yellow orange')

ax3.set_ylim(-70,7)
ax4.set_ylim(-70,7)
ax3.grid()

ax3.set_xlim(-0.1,8.1)

ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
plt.setp(ax1.get_xticklabels(), visible=False)

##### Plot R/L et alpha
ax5 = fig1.add_subplot(gs[1],sharex=ax1)
ax6 = fig1.add_subplot(gs[3],sharex=ax5)

rtoLratio = [[] for k in range(len(points_to_plot))]
for i,j in enumerate(points_to_plot):
    for k,l in enumerate(dim_nu[j,:,0]):
        rtoLratio[i].append( dim_nu[j,k,0] / dim_nu[j,k,2])
mean_rtoLratio = [mean(rtoLratio[i][:]) for i,j in enumerate(points_to_plot)]
for i,j in enumerate(points_to_plot):
    ax5.plot([plot_Udc[i]]*len(rtoLratio[i][:]),rtoLratio[i][:],'o',color='xkcd:azul',markerfacecolor='none')
#     ax6.plot(plot_Udc[i],alpha_dre[0][j],'o',color='xkcd:azul')
    
ax5.plot(plot_Udc,mean_rtoLratio,linestyle='',marker='o',color='xkcd:yellow orange',markerfacecolor='none')

ax6.set_xlabel('$U_{DC}$')
ax5.set_ylabel('R/L')
ax6.set_ylabel(r'$\alpha$')
# ax5.tick_params(axis='y', labelcolor='xkcd:azul')
# ax6.tick_params(axis='y', labelcolor='xkcd:yellow orange')

ax6.set_xlim(-0.1,8.1)
ax5.set_ylim(-0.05,0.6)
ax6.set_ylim(-1.65,0.35)
ax5.grid()
ax6.grid()

ax5.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
plt.setp(ax5.get_xticklabels(), visible=False)

tight_layout()

fig1.set_size_inches(11.69, 8.27)
fig1.tight_layout()
subplots_adjust(hspace=0.015,top=0.935)

# savefig(fname,dpi=300)

In [ ]:
ax = 0
ay = 0
az, qx, wx, wy, wz = [],[],[],[],[]
alpha_dre = zeros(shape(points_to_plot))

for i,j in enumerate(points_to_plot):
    az.append(8*mkappa*(C_e/m_Ca)*plot_Udc[i]/(d0**2*Omega**2))
    qx.append(-4*(C_e/m_Ca)*plot_Udc[i]/(r0**2*Omega**2))

    # frequences attendues
    wx.append(Omega/2*sqrt(qx[i]**2/2+ax-0.5*az[i])/(2*pi))
    #     wy.append(Omega/2*sqrt(qx[l]**2/2+ay-0.5*az[l])/(2*pi))
    wz.append(sqrt(2*mkappa*C_e*plot_Udc[i]/(m_Ca*d0**2))/(2*pi))


for k,l in enumerate(plot_Urf): # try
    print(j,k,l)
    alpha_dre[k][j] = [j]**2/wx[j][l]**2

In [ ]:
# histogram for Delta energy
# two families : detection and non detection
trig_threshold = 0.6
my_trigger = [ [ deltaEc[k][my_order[j]]*renorm*1000 for k in range(elem_0) if PMvar[k][my_order[j]]>trig_threshold] for j in range(elem_2) ]
# my_DE = [ [deltaEc[k,my_selection[l]]*renorm*1000 for k in range(elem_0)] for l in range(elem_2) ]
y_hist_detec = concatenate(( [my_trigger[my_order[j]] for j in range(elem_2)] ))

my_nontrigger = [ [ deltaEc[k][my_order[j]]*renorm*1000 for k in range(elem_0) if PMvar[k][my_order[j]]<trig_threshold] for j in range(elem_2) ]
# my_DE = [ [deltaEc[k,my_selection[l]]*renorm*1000 for k in range(elem_0)] for l in range(elem_2) ]
y_hist_nondetec = concatenate(( [my_nontrigger[my_order[j]] for j in range(elem_2)] ))


fig = plt.figure(num='Histogram Delta energy GMol', clear=True)
# plt.suptitle('Histogram $\Delta E$')
# gs = gridspec.GridSpec(2, 2, width_ratios=[3, 1]) 
##### PLOT DFluo et Proba détection
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212,sharex=ax1)
 # PMvar[k][my_selection]
my_bins = arange(-120,0,2.5)
ax1.set_title(r'Detection $\Delta F/F>0.6$',color='xkcd:leaf green')
ax1.hist(y_hist_detec,bins=my_bins ,color='xkcd:leaf green', edgecolor='black', linewidth=1.2)
ax2.set_xlabel(r'$\Delta E$ [meV]')
ax1.set_ylim(0,35)
ax1.grid()
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=10))
ax1.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=4))
ax1.annotate(f'{len(y_hist_detec)} cases', xy=(-123,33), xycoords='data',
    size=12, ha='left', va='top', color='xkcd:leaf green',
    bbox=dict(boxstyle='round', fc='white')) 

ax2.set_title(r'No detection $\Delta F/F<0.6$',color='xkcd:rouge')
ax2.hist(y_hist_nondetec,bins=my_bins ,color='xkcd:rouge', edgecolor='black', linewidth=1.2)
ax2.set_ylim(0,35)
ax2.grid()
ax2.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=4))
ax2.annotate(f'{len(y_hist_nondetec)} cases', xy=(-123,33), xycoords='data',
    size=12, ha='left', va='top', color='xkcd:rouge',
    bbox=dict(boxstyle='round', fc='white')) 

fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
subplots_adjust(hspace=0.22,top=0.94)

# savefig('hist_simu.png', dpi=300)

In [ ]:
# plot DE vs Udc but with red green colors

# my_order = [0,1,2,3,4,5,6,13,7,8,9,14,15,10,16,11,12] # 13,7,8,9,14,15,10,16,11,12

# Udc = [0.5,1,1.5,2,2.5,3,3.5,3.75,4,4.5,5,5.3,5.7,6,6.5,7,8]
# Urf = 64.61

mean_fluo = [mean(PMvar[:,j]) for j in my_order]

fig1 = plt.figure(num='Fluo var only_DE_colors',clear=True)

##### PLOT DFluo et Proba détection
ax1 = fig1.add_subplot(111)
ax2 = ax1.twinx()

left, bottom, width, height = [0.4145, 0.12, 0.45, 0.5] # cadre zoom
axins = fig1.add_axes([left, bottom, width, height])

my_color = ['xkcd:rouge', 'xkcd:leaf green']
l = 0
for j in range(len(Udc)):          
    # a,q attendu
    az.append(8*mkappa*(e/m)*Udc[j]/(d0**2*Omega**2))
    qx.append(-4*(e/m)*Urf/(r0**2*Omega**2))

    # frequences attendues
    wx.append(Omega/2*sqrt(qx[l]**2/2+ax-0.5*az[l])/(2*pi))
#     wy.append(Omega/2*sqrt(qx[l]**2/2+ay-0.5*az[l])/(2*pi))
    wz.append(sqrt(2*mkappa*e*Udc[j]/(m*d0**2))/(2*pi))
    
    for k in range(elem_0):
        trig = 0
        if PMvar[k][my_order[j]] > 0.6:
            sigdetect[j] +=1
            trig = 1
        alpha_dre[k][j] = wz[j]**2/wx[j]**2
        rtoLratio[k][j] = dim_nu[k,j,0] / dim_nu[k,j,2]
        ax1.plot(Udc[j],deltaEc[k][my_order[j]]*renorm*1000,'o',color=my_color[trig],markerfacecolor='none')
        axins.plot(Udc[j],deltaEc[k][my_order[j]]*renorm*1000,'o',color=my_color[trig],markerfacecolor='none')
    
    l+=1


mean_deltaEc = [mean(deltaEc[:,j])*renorm*1000 for j in my_order]
        
ax2.plot(Udc,mean_deltaEc,linestyle=':',marker='>',color='xkcd:yellow orange',markerfacecolor='none')
ax2.annotate('$E_{GMol}$=50 eV', xy=(1,-5), xycoords='data',
    size=10, ha='center', va='top', color='xkcd:azul',
    bbox=dict(boxstyle='round', fc='none',edgecolor='xkcd:azul')) 

ax1.set_xlabel('$U_{DC}$')
ax1.set_ylabel('$\Delta E$ [meV]', color='xkcd:azul')
ax2.set_ylabel('mean $\Delta E$ [meV]', color='xkcd:yellow orange')
ax1.tick_params(axis='y', labelcolor='xkcd:azul')
ax2.tick_params(axis='y', labelcolor='xkcd:yellow orange')

ax1.set_ylim(-0.10*1000,1)
ax2.set_ylim(-0.10*1000,1)
ax1.grid()

ax2.set_xlim(-0.1,8.5)

ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
ax1.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=10))

x1, x2, y1, y2 = 3.25, 8.25, -21, -2.5 # limites zoom
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.grid()

mark_inset(ax1, axins, loc1=2, loc2=1, fc="none", ec="0.4")
# axins.invert_yaxis()
plt.setp(axins.get_xticklabels(), visible=False)
plt.setp(axins.get_yticklabels(), fontsize=10)


fig1.set_size_inches(11.69, 8.27)
fig1.tight_layout()

## Stopping power

In [ ]:
deltaEc

In [ ]:
m_GM = 1e4*1.66054e-27
renorm = 0.5*m_GM/C_e
GMol_energy_loss = mean(deltaEc,axis=1)*renorm # eV
print(GMol_energy_loss)

plot_EGmol = [1000,800,600,400,200,100,50,10,8,6,4,2,1,20,30,40,60,70,80,90,150,300,500,700,900]
# plot_EGmol = [1000,800,600,400,200,100,50,10,8,6,4,2,1,20,40,70,90,300,700]

In [ ]:
plot_EGmol_indiv, deltaEc_indiv = zip(*sorted(zip(plot_EGmol, deltaEc)))
plot_EGmol, GMol_energy_loss = zip(*sorted(zip(plot_EGmol, GMol_energy_loss)))

In [ ]:
fname = 'stopping_power'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
# title(all_subdir[0][:-15],fontsize=12)

# im = ax.scatter(plot_EGmol,GMol_energy_loss*1000,
#         marker='o',s=100,edgecolor='black', cmap='rainbow')
im = ax.plot(plot_EGmol,np.multiply(GMol_energy_loss,1000),
        marker='+',ls=':',ms=15,mew=2,mec='xkcd:rouge',lw=2,color='xkcd:gray')
for k in range(len(deltaEc)):
    ax.scatter([plot_EGmol_indiv[k]]*len(deltaEc_indiv[0]),np.multiply(deltaEc_indiv[k],1000*renorm))
ax.set_xlabel(r'$E_{GMol}^i$ [eV]')
ax.set_ylabel(r'$\Delta E_{GMol}$ [meV]')
ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=4))

ax.set_xlim(-1,71)
ax.set_ylim(-25,1)
# ax.grid()
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'_focus_10e4.png',dpi=300)

In [ ]:
fname = 'stopping_power_zoom'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
# title(all_subdir[0][:-15],fontsize=12)

# im = ax.scatter(plot_EGmol,GMol_energy_loss*1000,
#         marker='o',s=100,edgecolor='black', cmap='rainbow')
im = ax.plot(plot_EGmol,np.multiply(GMol_energy_loss,1000),
        marker='+',ls=':',ms=15,mew=2,mec='xkcd:rouge',lw=2,color='xkcd:gray')
for k in range(len(deltaEc)):
    ax.scatter([plot_EGmol_indiv[k]]*len(deltaEc_indiv[0]),np.multiply(deltaEc_indiv[k],1000*renorm))
ax.set_xlabel(r'$E_{GMol}^i$ [eV]')
ax.set_ylabel(r'$\Delta E_{GMol}$ [meV]')
ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=4))

ax.set_xlim(-1,71)
ax.set_ylim(-25,1)

# ax.grid()
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'_focus_10e4.png',dpi=300)

# Trajectory
Trajectoire GMol

In [ ]:
for k, address in enumerate(all_subdir):
    onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
    # print(sort(onlyfiles))
    my_file = '{}/xva{}'.format(address,sort(onlyfiles)[0].strip('.dat')[4:])
    r_LC,v_LC,a_LC = load_xyz_init_bin_DP(my_file)

In [ ]:
my_file

In [ ]:
plot_EGmol = [50]
actual_condition = 0

In [ ]:
cloud_condition = all_subdir[0][myslashpos[-2]+1:myslashpos[-1]]
temperature_files = []
trajectory_files = []
for k, address in enumerate(all_subdir):
    #if 'EG' in cloud_condition:
    #actual_condition = int( re.sub('[a-zA-Z]+','',cloud_condition) )
    sort_cond = plot_EGmol[actual_condition]
    temp_file_name = [f for f in listdir(address) if isfile(join(address, f)) and "Temp_SimuType4" in f and str(sort_cond) in f]
    temperature_files.append(temp_file_name)
    trj_file_name = [f for f in listdir(address) if isfile(join(address, f)) and "trj_SimuType4" in f and str(sort_cond) in f]
    trajectory_files.append(trj_file_name)

In [ ]:
# temp_size = len(load_trj(os.path.join(all_subdir[0],trajectory_files[0][0]).strip('.bin'))[0])
t, r_Gmol, v_Gmol, a_Gmol = [[], [],[],[]]
tt, PM   = [[],[]]
T_CM, T_aux = [[],[]]
all_space = []
all_temp = []
for j,k in enumerate(trajectory_files): # try
    print('>',j,'-',k)
    all_space = (load_trj(os.path.join(all_subdir[j],k[0].strip('.bin')))) # t[j], r_x[j],r_y[j],r_z[j], v_x[j],v_y[j],v_z[j], a_x[j],a_y[j],a_z[j]
    t.append(all_space[0])
    r_Gmol.append(all_space[1:4])
    v_Gmol.append(all_space[4:7])
    a_Gmol.append(all_space[7:10])
for j,k in enumerate(temperature_files): # try
    print('>',j,'-',k)
    try:
        all_temp = (load_T_and_PM_simu(os.path.join(all_subdir[j],k[0].strip('.dat'))))
    except:
        all_temp = [0,[0,0,0],[0,0,0],0]
    tt.append(all_temp[0])
    T_CM.append(all_temp[1])
    T_aux.append(all_temp[2])
    PM.append(all_temp[3])

# Ion cloud
Densité tout le tralala

In [ ]:
Gamma_Bussman

In [ ]:
cm = pylab.get_cmap('gist_rainbow')
Gamma_Bussman = [[] for k in enumerate(trajectory_files)]
T = [[] for k in enumerate(trajectory_files)]
aws = []
q_state = 1
for k,l in enumerate(trajectory_files):
    l=l[0]
    le_Urf = int(l[l.find('Vrf')+3:l.find('_Udc')])
    for m,Temp in enumerate(T_aux[k]):
        try:
            T[k].append(mean(Temp))
        except:
            T[k] = 0
        Psirad = 0.5*q_state*C_e/m_Ca/r0**4/Omega**2*le_Urf**2
        aws = ( 3*q_state*C_e / (8*np.pi * eps0 * Psirad) )**(1/3) # aws.append( ( 3*q_state*C_e / (8*np.pi * eps0 * Psirad) )**(1/3) )
        print(aws)
        Gamma_Bussman[k].append( q_state*C_e**2 / (4*np.pi*eps0*kb*T[k][m]*aws) )
        # print(le_Urf,Gamma_Bussman[k])
    
fname = 'gamma_selon_T'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
#title(all_subdir[0][:-15],fontsize=12)

pouet = 80
x = T[pouet]
y = Gamma_Bussman[pouet]
im = ax.scatter(x,y,
        marker='+',s=20,lw=1,color=cm(i/13))
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel(r'T [K]')
ax.set_ylabel(r'Gamma')
ax.set_xlim(0.25e-3,0.5)
ax.set_ylim(1,10000)
ax.grid()      
    # ax.grid()
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()

In [ ]:
Gamma_Bussman = []
aws = []
q_state = 1
for k,l in enumerate(trajectory_files):
    T = mean(T_aux[k][0])
    l=l[0]
    le_Urf = int(l[l.find('Vrf')+3:l.find('_Udc')])
    Psirad = 0.5*q_state*C_e/m_Ca/r0**4/Omega**2*le_Urf**2
    aws.append( ( 3*q_state*C_e / (8*pi * eps0 * Psirad) )**(1/3) )
    Gamma_Bussman.append( q_state*C_e**2 / (4*np.pi*eps0*kb*T*aws[k]) )
    print(le_Urf,Gamma_Bussman[k])

In [ ]:
T_aux[k][5]

In [ ]:
v_ca[0]

#  Some calculations about physics

In [ ]:
# comparing amplitude RF mvmnt to Thermal mvmnt

to_calc = 4
T = 6e-4

Er = 2*Urf*mean(dim_nu[:,my_order[to_calc],0])/r0

RF_amp = C_e*Er / (m_Ca*Omega**2)
Th_amp = sqrt(kb*T**2/ (m_Ca*wx[my_order[to_calc]]**2) )

print(Th_amp, RF_amp)

In [ ]:
frac_E0_Ecr = 50
wz_LC = 2*pi*90806.9982303
rz_0 = -15e-3

wz2 = wz_LC**2 * m_Ca / m_GM

L     = 0.014  #(demi-longueur piège obtenue avec SIMION)
eta = 1. / (4.*(sqrt(10) + 5.)) # (~0.0306 je soupçonne cette valeur d'être le facteur d'écrantage)

Ep0   = wz2*eta*L**2*m_GM*(1- exp(rz_0**2 / (2*eta*L**2)) )
Ek_init = C_e * frac_E0_Ecr - Ep0

print(frac_E0_Ecr,Ep0/C_e,Ek_init/C_e)

## Distance moyenne entre les ions Ca$^+$

In [ ]:
dist_ions = zeros(shapevar)
dist_ions_theo = zeros(shapevar)
V = zeros(shapevar)
V_theo = zeros(shapevar)
rho = zeros(shapevar)
k = 3
for k in range(elem_0):
    for j in range(elem_2):     # Udc
        rho[k][j] = 2*m*eps0/e**2*(wx[j]*2*pi)**2 # "théorique" on s'en fout
        V_theo[k][j] = N/rho[k][j]
        V[k][j] = 4/3*pi*dim_nu[k][my_order[j]][2]*1e-3 * ( dim_nu[k][my_order[j]][0]*1e-3 )**2
    #     rho[j] = N/V_moy[j] # donné par le volume et le nombre d'ions
    #     dist_ions[j] = ( 1 / (V_moy[j]*rho[j]) ) ** (-1/3) # en µm
        dist_ions[k][j] = (V[k][j]/N)**(1/3)
        dist_ions_theo[k][j] = (V_theo[k][j]/N)**(1/3)

In [ ]:
max_HC_trj = zeros((elem_0,elem_2,3))
for j in range(elem_2):     # Udc
    print('j - k')
    for k in range(elem_0): # try
        max_HC_trj[k,j,0] = max(abs( r_x[k][my_order[j]] ))
        max_HC_trj[k,j,1] = max(abs( r_y[k][my_order[j]] ))
        max_HC_trj[k,j,2] = max(abs( r_z[k][my_order[j]] ))

In [ ]:
fig = plt.figure(num='Dist ions et GMol',clear=True)
# gs = gridspec.GridSpec(2, 2, width_ratios=[8, 4])   gs[0]
##### PLOT DFluo et Proba détection
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212,sharex = ax1)
# ax3 = fig.add_subplot(gs[1])
# ax4 = fig.add_subplot(gs[3], projection='3d')

for j in range(elem_2):
    for k in range(elem_0):
        Vtmp = 4/3*pi*dim_nu[k][my_order[j]][2]*1e-3 * ( dim_nu[k][my_order[j]][0]*1e-3 )**2
        dist_ionstmp = (Vtmp/N)**(1/3)
        ax1.plot(Udc[j],dist_ionstmp*1e6,'o',color='xkcd:azul',markerfacecolor='none')
    ax1.plot(Udc[j],mean(dist_ions[:,j])*1e6,'o',marker='8',mew=2,mfc='None',mec='xkcd:yellow orange')
# ax1.set_xlabel(r'$U_{DC}$ [V]')
ax1.set_ylabel(r'$d_{moy}$ Ca$^+$ [µm]')
ax1.grid()

for j in range(elem_2):
    for k in range(elem_0):
        r_sortie = sqrt(max_HC_trj[k,j,0]**2 + max_HC_trj[k,j,1] **2 )
        ax2.plot(Udc[j],r_sortie*1e9,'o',color='xkcd:azul',markerfacecolor='none')
    r_sortie = sqrt(mean(max_HC_trj[:,j,0])**2 + mean(max_HC_trj[:,j,1]) **2 )
    ax2.plot(Udc[j],r_sortie*1e9,marker='8',mew=2,mfc='None',mec='xkcd:yellow orange')

#     ax2.plot(Udc[j],mean(max_HC_trj[:,j,1])*1e9,color='xkcd:azul',marker='+')
#     ax2.plot(Udc[j],mean(max_HC_trj[:,j,2])*1e9,color='xkcd:brown',marker='+')
ax2.set_xlabel(r'$U_{DC}$ [V]')
ax2.set_ylabel('r sortie max GMol [nm]')
ax2.grid()

ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
plt.setp(ax1.get_xticklabels(), visible=False)

fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
subplots_adjust(hspace=0.015)

In [ ]:
# Dimensions nuage
k = 3
j = 3
for k in range(elem_0):
    print(k,dim_nu[k][j]*1e3)
print(mean(dim_nu[k][j][0])*1e3)
print(mean(dim_nu[k][j][1])*1e3)
print(mean(dim_nu[k][j][2])*1e3)

In [ ]:
dist_ions*1e6

In [ ]:
dist_ions_theo * 1e6

# Ecriture fichier
Garder les principales variables dans un fichier texte

In [ ]:
float_formatter = lambda x: '{:.04}'.format(x)

In [ ]:
# name=['Uend ', 'ax   ','az   ','qx   ','kappa','Udc  ','Urf  ','aax  ','aaz  ','aqx  ','awx  ','awz  ','aUdc ','aUend']
# var=[Uend, ax, az, qx, kappa, Udc, Urf, aax, aaz, aqx, awx, awz, aUdc, aUend]


f=open("simu_variables.txt","w+")
f.write('Python3.7 %s\n' % datetime.datetime.now())
f.write('Simulations collisions GMol -> Ca+ \n\n\n')

f.write('> Répertoire de travail\n')
f.write('%s\n\n' % work_rep)

f.write('> Conditions de simulation\n')
f.write(f'N_ions E_GMol\n')
f.write(f'{N} {e_GMol:>4}eV\n\n')

f.write(f'> Nombre total de simulations : {cond_number}\n')
f.write(f'{cond_two_name} : {elem_2} élément(s)\n')
f.write(f'{[ int(x) for x in cond_two]}\n')
f.write(f'{[ Udc[x] for x in my_order ]} V\n')
f.write(f'{cond_one_name} : {elem_1} élément(s)\n')
f.write(f'{[ int(x) for x in cond_one]}\n')
f.write(f'{[ int(x) for x in [Urf]]} V\n')
f.write(f'{cond_zero_name} : {elem_0} élément(s)\n')
f.write(f'{try_subdir} \n\n')

f.write(f'> X, Y, Z : Demi-axes moyens du nuage Ca+ [µm]\n')
f.write(f'> d_moy : Distance moyenne entre les ions du nuage Ca+ [µm]\n')
f.write(f' Udc  :     X      Y      Z      d_moy\n')
for j in range(elem_2):
    f.write(f'{Udc[j]:.2f}  :  {[ float(float_formatter(mean(dim_nu[:,my_order[j],x])*1000)) for x in [0,1,2]]}  {dist_ions[j]*1e6:.3f}\n')
    
# f.write(f'> Molécule géante - position max\n')
# f.write(f' Udc  :    rx[nm]      ry[nm]     rz[mm]\n')
# for j in range(elem_2):
#     f.write(f'{Udc[j]:.2f}  :  {[ float_formatter(r_x[my_order[j]]), r_y[my_order[j]], r_z[my_order[j]] ]}\n')

f.close()

In [ ]:
my_q = [0.5,0.5+0.05/3,0.5+2*0.05/3,0.55,0.55+0.05/3,0.55+2*0.05/3,0.6,0.6+0.05/3,0.6+2*0.05/3, 0.65, 0.65+0.05/3, 0.65+0.1/3, 0.7] # [0.6, 0.625, 0.650, 0.675, 0.7]
print(my_q)
coucou = []
figure('pouet')
print('q','   ','Vrf')
for k,l in enumerate(my_q):
    coucou.append(tick_function_q(l))
    print(f'{l:2.5f}',f'{tick_function_q(l):2.2f}')
scatter(my_q,coucou)